In [1]:
import json
import numpy as np
import matplotlib.pyplot as plt

import torch

from nerf_model import NerfModel
from nerf_utils import get_rays

In [2]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3080'